In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
df_notes = pd.read_csv('/scratch/kh2383/MIMIC/1.4/NOTEEVENTS.csv')

/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/ipython-6.2.1-py3.6.egg/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
df_adm = pd.read_csv('/scratch/kh2383/MIMIC/1.4/ADMISSIONS.csv')
df_patients = pd.read_csv('/scratch/kh2383/MIMIC/1.4/PATIENTS.csv')
df_icus = pd.read_csv('/scratch/kh2383/MIMIC/1.4/ICUSTAYS.csv')

In [4]:
df_icus.INTIME = pd.to_datetime(df_icus.INTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

In [5]:
len(df_icus)

61532

In [6]:
len(df_icus.HADM_ID.unique())

57786

In [7]:
df_icu_first = df_icus.sort_values(['INTIME']).groupby('HADM_ID').head(1)

In [8]:
len(df_icu_first)

57786

In [9]:
len(df_adm)

58976

In [10]:
df_adm = pd.merge(df_adm, df_icu_first[['HADM_ID','LOS']], on = 'HADM_ID')

In [11]:
df_adm.columns.values

array(['ROW_ID', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME',
       'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'LOS'], dtype=object)

In [12]:
df_patients.columns.values

array(['ROW_ID', 'SUBJECT_ID', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP',
       'DOD_SSN', 'EXPIRE_FLAG'], dtype=object)

In [13]:
df = pd.merge(df_adm, df_patients, on = ['SUBJECT_ID'])

In [14]:
len(df)

57786

In [15]:
df_notes_physician = df_notes[df_notes.CATEGORY == 'Physician ']

In [16]:
len(df_notes_physician.HADM_ID.unique())

8984

In [17]:
cohort = df_notes_physician.HADM_ID.unique()

In [18]:
len(cohort)

8984

In [19]:
df = df[df.HADM_ID.isin(cohort)]

In [20]:
len(df)

8981

In [21]:
df_info = df[['SUBJECT_ID', 'HADM_ID', 'ADMITTIME','DISCHTIME', 'DOB', 'DOD_HOSP','LOS']].drop_duplicates().reset_index(drop = True)

In [22]:
df_info.DOD = pd.to_datetime(df_info.DOD_HOSP, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_info.DOB = pd.to_datetime(df_info.DOB, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_info.ADMITTIME = pd.to_datetime(df_info.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_info.DISCHTIME = pd.to_datetime(df_info.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

/home/kh2383/pytorch-gpu/py3.6.3/lib/python3.6/site-packages/ipykernel_launcher.py:1: UserWarning: Pandas doesn't allow columns to be created via a new attribute name - see https://pandas.pydata.org/pandas-docs/stable/indexing.html#attribute-access
  """Entry point for launching an IPython kernel.


In [23]:
df_info = df_info.assign(DEATH_HOSP = [1 if i else 0 for i in (df_info.DOD_HOSP.notnull())])

In [24]:
df_info.DEATH_HOSP.value_counts()

0    6276
1    2705
Name: DEATH_HOSP, dtype: int64

In [25]:
len(df_info)

8981

In [26]:
df_info = df_info.assign(AGE = ((df_info['ADMITTIME']-df_info['DOB']).dt.total_seconds()/(60*60*24*365)))
df_info = df_info.assign(LOS_HOS = ((df_info['DISCHTIME']-df_info['ADMITTIME']).dt.total_seconds()/(60*60*24)))
df_info = df_info.assign(LOS_7 = [1 if i else 0 for i in (df_info.LOS >= 7)])

In [27]:
df_info.LOS_7.value_counts()

0    7935
1    1046
Name: LOS_7, dtype: int64

In [28]:
df_info = df_info.assign(Label = [1 if i else 0 for i in ((df_info.LOS_7 == 1) | (df_info.DEATH_HOSP == 1))])

In [29]:
df_info.Label.value_counts()

0    5637
1    3344
Name: Label, dtype: int64

In [30]:
# what is care limitaion?
df_info = df_info[df_info.AGE>=18]

In [31]:
len(df_info.reset_index(drop = True))

8469

In [32]:
df_info = df_info[df_info.LOS_HOS >= 2].reset_index(drop = True) # 48 hours LOS

In [33]:
len(df_info)

7718

In [34]:
df_notes_interest = df_notes[df_notes.CATEGORY.isin(['Physician ', 'Nursing', 'Nursing/other'])]
df_notes_interest.CHARTTIME = pd.to_datetime(df_notes_interest.CHARTTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')

/share/apps/python3/3.6.3/intel/lib/python3.6/site-packages/pandas-0.22.0-py3.6-linux-x86_64.egg/pandas/core/generic.py:3643: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self[name] = value


In [35]:
df_notes_info = pd.merge(df_notes_interest, df_info, on = 'HADM_ID')

In [36]:
len(df_notes_info.HADM_ID.unique())

7718

In [37]:
len(df_notes_info)

335596

In [38]:
# within 48 hours of notes
df_notes_info = df_notes_info[(df_notes_info.CHARTTIME - df_notes_info.ADMITTIME).dt.total_seconds()/(60*60) < 48]

In [39]:
len(df_notes_info.HADM_ID.unique())

6521

In [40]:
# not null notes
df_notes_info = df_notes_info[df_notes_info.TEXT.notnull()]

In [41]:
len(df_notes_info.HADM_ID.unique())

6521

In [42]:
# remove iserror notes
df_notes_info = df_notes_info[df_notes_info.ISERROR.isnull()]

In [43]:
len(df_notes_info.HADM_ID.unique())

6521

In [44]:
df_final = df_notes_info[['HADM_ID', 'TEXT', 'CHARTTIME','CATEGORY','Label','DEATH_HOSP','LOS']].drop_duplicates().reset_index(drop = True)

In [45]:
len(df_final.HADM_ID.unique())

6521

In [46]:
df_final[['HADM_ID','Label']].drop_duplicates().reset_index(drop = True).Label.value_counts()

0    4180
1    2341
Name: Label, dtype: int64

In [47]:
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk import PorterStemmer
import nltk
nltk.download('wordnet')

stemmer = SnowballStemmer("english")
lemmatizer = WordNetLemmatizer()
def lemmatize_stemming(text):
    return ' '.join([stemmer.stem(i) for i in (lemmatizer.lemmatize(j) for j in text.split())])

[nltk_data] Downloading package wordnet to /home/kh2383/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [48]:
import re
import string

def preprocess1(x):
    y=re.sub('\\[(.*?)\\]','',x) #remove de-identified brackets
    y=re.sub('[0-9]+\.','',y) #remove 1.2. since the segmenter segments based on this
    y=re.sub('dr\.','doctor',y)
    y=re.sub('m\.d\.','md',y)
    y=re.sub('--|__|==','',y)
    
    # remove punctuation, digits, spaces
    #y = y.translate(str.maketrans("", "", string.punctuation))
    y = y.translate(str.maketrans("", "", string.digits))
    y = " ".join(y.split())
    return y

def preprocessing_note(df_less_n): 
    df_less_n['TEXT']=df_less_n['TEXT'].fillna(' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\n',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].str.replace('\r',' ')
    df_less_n['TEXT']=df_less_n['TEXT'].apply(str.strip)
    df_less_n['TEXT']=df_less_n['TEXT'].str.lower()

    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: preprocess1(x))
    df_less_n['TEXT']=df_less_n['TEXT'].apply(lambda x: lemmatize_stemming(x))

    df_less_n['# of tokens'] = df_less_n['TEXT'].str.split().str.len()
    df_less_n = df_less_n[df_less_n['# of tokens'] > 5]
    return df_less_n


In [57]:
df_final[df_final.CATEGORY == 'Physician ']['TEXT'].to_csv('./physician_raw.csv')

In [59]:
df_final[df_final.CATEGORY.isin(['Nursing', 'Nursing/other'])]['TEXT'].to_csv('./nursing_raw.csv')

In [61]:
df_use[df_use.CATEGORY == 'Physician ']['TEXT'].to_csv('./physician_preprocessed.csv')

In [62]:
df_use[df_use.CATEGORY.isin(['Nursing', 'Nursing/other'])]['TEXT'].to_csv('./nursing_preprocessed.csv')

In [58]:
df_use = preprocessing_note(df_final)

In [164]:
!mkdir /scratch/kh2383/Mortality/data/fold10

In [165]:
# create one untouchable test set first (instead of create five) just to enlarge the pretraining notes size
df_HADM_ID = df_use.HADM_ID.drop_duplicates().reset_index(drop = True)
test_id = df_HADM_ID.sample(frac = 0.1, replace = False, random_state = 1)
test = df_use[df_use.HADM_ID.isin(test_id.values)]
test.reset_index(drop = True).to_csv('/scratch/kh2383/Mortality/data/test.csv')

train_val_id = df_HADM_ID[~df_HADM_ID.index.isin(test_id.index)]

for i in tqdm(range(6,11)):
    val_id = train_val_id.sample(frac = 1/9, replace = False, random_state = i)
    train_id = train_val_id[~train_val_id.index.isin(val_id.index)]
        
    train = df_use[df_use.HADM_ID.isin(train_id.values)]
    val = df_use[df_use.HADM_ID.isin(val_id.values)]
    train_val = df_use[df_use.HADM_ID.isin(train_val_id)]
    
    path = '/scratch/kh2383/Mortality/data/fold'+str(i)
    train.reset_index(drop = True).to_csv(path + '/train.csv')
    val.reset_index(drop = True).to_csv(path + '/val.csv')
    train_val.reset_index(drop = True).to_csv(path + '/train_val.csv')

100%|██████████| 5/5 [01:02<00:00, 12.46s/it]


In [83]:
df_use[df_use.CATEGORY == 'Physician '].reset_index(drop = True).to_csv('/scratch/kh2383/Mortality/data/physician.csv')
df_use[df_use.CATEGORY.isin(['Nursing', 'Nursing/other'])].reset_index(drop = True).to_csv('/scratch/kh2383/Mortality/data/nursing.csv')

In [84]:
df_use.CATEGORY.value_counts()

Nursing          61569
Physician        41478
Nursing/other      300
Name: CATEGORY, dtype: int64

In [119]:
len(df_use.HADM_ID.unique())

6521

In [85]:
from tableone import TableOne

In [88]:
HADM_ID_use = df_use.HADM_ID.unique()

In [91]:
df_info = df

In [94]:
df_info.DOD = pd.to_datetime(df_info.DOD_HOSP, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_info.DOB = pd.to_datetime(df_info.DOB, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_info.ADMITTIME = pd.to_datetime(df_info.ADMITTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_info.DISCHTIME = pd.to_datetime(df_info.DISCHTIME, format = '%Y-%m-%d %H:%M:%S', errors = 'coerce')
df_info = df_info.assign(DEATH_HOSP = [1 if i else 0 for i in (df_info.DOD_HOSP.notnull())])
df_info = df_info.assign(AGE = ((df_info['ADMITTIME']-df_info['DOB']).dt.total_seconds()/(60*60*24*365)))
df_info = df_info.assign(LOS_HOS = ((df_info['DISCHTIME']-df_info['ADMITTIME']).dt.total_seconds()/(60*60*24)))
df_info = df_info.assign(LOS_7 = [1 if i else 0 for i in (df_info.LOS >= 7)])
df_info = df_info.assign(Label = [1 if i else 0 for i in ((df_info.LOS_7 == 1) | (df_info.DEATH_HOSP == 1))])

In [95]:
df_info.columns.values

array(['ROW_ID_x', 'SUBJECT_ID', 'HADM_ID', 'ADMITTIME', 'DISCHTIME',
       'DEATHTIME', 'ADMISSION_TYPE', 'ADMISSION_LOCATION',
       'DISCHARGE_LOCATION', 'INSURANCE', 'LANGUAGE', 'RELIGION',
       'MARITAL_STATUS', 'ETHNICITY', 'EDREGTIME', 'EDOUTTIME',
       'DIAGNOSIS', 'HOSPITAL_EXPIRE_FLAG', 'HAS_CHARTEVENTS_DATA', 'LOS',
       'ROW_ID_y', 'GENDER', 'DOB', 'DOD', 'DOD_HOSP', 'DOD_SSN',
       'EXPIRE_FLAG', 'DEATH_HOSP', 'AGE', 'LOS_HOS', 'LOS_7', 'Label'], dtype=object)

In [96]:
df_info_ = df_info[df_info.HADM_ID.isin(HADM_ID_use)]

In [97]:
len(df_info_)

6521

In [114]:
df_info_.GENDER.value_counts()

M    3703
F    2818
Name: GENDER, dtype: int64

In [110]:
columns = ['ADMISSION_TYPE', 'GENDER', 'ETHNICITY', 'LOS','LOS_HOS','AGE', 'DEATH_HOSP','LOS_7','Label']
categorical = ['ADMISSION_TYPE','ETHNICITY','GENDER','DEATH_HOSP','LOS_7','Label']
groupby = 'Label'

In [111]:
mytable = TableOne(df_info_, columns, categorical, groupby, pval = True)

/home/kh2383/pytorch-gpu/py3.6.3/lib/python3.6/site-packages/tableone.py:638: UserWarning: No p-value was computed for ETHNICITY due to the low number of observations.
  warnings.warn('No p-value was computed for {} due to the low number of observations.'.format(v))


In [112]:
mytable

Grouped by Label                                                                               
                                                                            isnull             0             1    pval                                      ptest
variable       level                                                                                                                                             
n                                                                                           4180          2341                                                   
ADMISSION_TYPE ELECTIVE                                                          0    742 (17.8)     157 (6.7)  <0.001                                Chi-squared
               EMERGENCY                                                             3390 (81.1)   2163 (92.4)                                                   
               URGENT                                                                   48 (1.1)      21 (0.9)                                                   
GENDER         F                                                                 0   1809 (43.3)   1009 (43.1)   0.911                                Chi-squared
               M                                                                     2371 (56.7)   1332 (56.9)                                                   
ETHNICITY      AMERICAN INDIAN/ALASKA NATIVE                                     0       4 (0.1)       2 (0.1)   0.004  Chi-squared (warning: expected count < 5)
               AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNI...                                      1 (0.0)                                                   
               ASIAN                                                                    56 (1.3)      28 (1.2)                                                   
               ASIAN - ASIAN INDIAN                                                     16 (0.4)       2 (0.1)                                                   
               ASIAN - CAMBODIAN                                                         1 (0.0)                                                                 
               ASIAN - CHINESE                                                          25 (0.6)      16 (0.7)                                                   
               ASIAN - FILIPINO                                                          1 (0.0)       2 (0.1)                                                   
               ASIAN - JAPANESE                                                          1 (0.0)       1 (0.0)                                                   
               ASIAN - KOREAN                                                            3 (0.1)       1 (0.0)                                                   
               ASIAN - OTHER                                                             1 (0.0)       1 (0.0)                                                   
               ASIAN - THAI                                                              1 (0.0)                                                                 
               ASIAN - VIETNAMESE                                                        6 (0.1)       3 (0.1)                                                   
               BLACK/AFRICAN                                                             5 (0.1)       3 (0.1)                                                   
               BLACK/AFRICAN AMERICAN                                                 424 (10.1)    240 (10.3)                                                   
               BLACK/CAPE VERDEAN                                                       22 (0.5)      16 (0.7)                                                   
               BLACK/HAITIAN                                                            10 (0.2)       2 (0.1)                                                   
               C

In [113]:
groupby = ''
TableOne(df_info_, columns, categorical, groupby)

isnull      overall
variable       level                                                                 
n                                                                                6521
ADMISSION_TYPE ELECTIVE                                                0   899 (13.8)
               EMERGENCY                                                  5553 (85.2)
               URGENT                                                        69 (1.1)
GENDER         F                                                       0  2818 (43.2)
               M                                                          3703 (56.8)
ETHNICITY      AMERICAN INDIAN/ALASKA NATIVE                           0      6 (0.1)
               AMERICAN INDIAN/ALASKA NATIVE FEDERALLY RECOGNI...             1 (0.0)
               ASIAN                                                         84 (1.3)
               ASIAN - ASIAN INDIAN                                          18 (0.3)
               ASIAN - CAMBODIAN                                              1 (0.0)
               ASIAN - CHINESE                                               41 (0.6)
               ASIAN - FILIPINO                                               3 (0.0)
               ASIAN - JAPANESE                                               2 (0.0)
               ASIAN - KOREAN                                                 4 (0.1)
               ASIAN - OTHER                                                  2 (0.0)
               ASIAN - THAI                                                   1 (0.0)
               ASIAN - VIETNAMESE                                             9 (0.1)
               BLACK/AFRICAN                                                  8 (0.1)
               BLACK/AFRICAN AMERICAN                                      664 (10.2)
               BLACK/CAPE VERDEAN                                            38 (0.6)
               BLACK/HAITIAN                                                 12 (0.2)
               CARIBBEAN ISLAND                                               4 (0.1)
               HISPANIC OR LATINO                                           187 (2.9)
               HISPANIC/LATINO - CENTRAL AMERICAN (OTHER)                     3 (0.0)
               HISPANIC/LATINO - COLOMBIAN                                    1 (0.0)
               HISPANIC/LATINO - CUBAN                                        6 (0.1)
               HISPANIC/LATINO - DOMINICAN                                   15 (0.2)
               HISPANIC/LATINO - GUATEMALAN                                   7 (0.1)
               HISPANIC/LATINO - MEXICAN                                      5 (0.1)
               HISPANIC/LATINO - PUERTO RICAN                                48 (0.7)
               HISPANIC/LATINO - SALVADORAN                                   4 (0.1)
               MIDDLE EASTERN                                                 5 (0.1)
               MULTI RACE ETHNICITY                                          11 (0.2)
               NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER                      1 (0.0)
               OTHER                                                         98 (1.5)
               PATIENT DECLINED TO ANSWER                                    60 (0.9)
               PORTUGUESE                                                    17 (0.3)
               UNABLE TO OBTAIN                                             146 (2.2)
               UNKNOWN/NOT SPECIFIED                                        113 (1.7)
               WHITE                                                      4850 (74.4)
               WHITE - BRAZILIAN                                             11 (0.2)
               WHITE - EASTERN EUROPEAN                                       3 (0.0)
               WHITE - OTHER EUROPEAN                                        11 (0.2)
               WHITE - RUSSIAN                                               21 (0.3)
LOS                     

In [139]:
df_fin = df_final

In [151]:
df_final = df_fin[df_fin.Label == 0]

In [152]:
np.mean(df_final[df_final.CATEGORY == 'Physician '].groupby('HADM_ID').TEXT.agg(len).values)

5.940747935891209

In [153]:
np.std(df_final[df_final.CATEGORY == 'Physician '].groupby('HADM_ID').TEXT.agg(len).values)

3.8018407167375874

In [154]:
np.mean(df_final[df_final.CATEGORY.isin(['Nursing','Nursing/other'])].groupby('HADM_ID').TEXT.agg(len).values)

9.1329646549651358

In [155]:
np.std(df_final[df_final.CATEGORY.isin(['Nursing','Nursing/other'])].groupby('HADM_ID').TEXT.agg(len).values)

4.5985631464333094

In [156]:
word_count = pd.DataFrame(df_final[df_final.CATEGORY.isin(['Nursing','Nursing/other'])].groupby(['HADM_ID','Label']).TEXT.apply(' '.join)).TEXT.str.split().str.len() 

In [157]:
print(np.mean(word_count.values))
print(np.std(word_count.values))

2181.99350805
1609.68578375


In [158]:
word_count = pd.DataFrame(df_final[df_final.CATEGORY == 'Physician '].groupby(['HADM_ID','Label']).TEXT.apply(' '.join)).TEXT.str.split().str.len() 

In [159]:
print(np.mean(word_count.values))
print(np.std(word_count.values))

4241.26566294
3130.57546684
